In [36]:
import wget
URL = "http://ovne.org/download/Customer-Churn-Records.csv"
response = wget.download(URL, "Customer-Churn-Records.csv")

In [37]:
import pandas as pd
df = pd.read_csv('Customer-Churn-Records.csv', header = 0, na_values='NA', comment='\t', sep=',', skipinitialspace=True)
df.drop(columns=['RowNumber', 'CustomerId', 'Surname', 'Satisfaction Score', 'Card Type', 'Point Earned'], inplace=True, axis = 1)

In [38]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Complain
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,1
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,0


In [39]:
# convert France = 0 , Spain = 1, "Germany" = 2
df['Geography'] = df['Geography'].replace({"France": 0, "Spain": 1, "Germany": 2})
# convert Male = 1, Female = 0
df['Gender'] = df['Gender'].replace({"Male": 1, "Female": 0})

/var/folders/bj/4zbr51sj4fsbvtllcchl3wzc0000gn/T/ipykernel_1780/1114702664.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Geography'] = df['Geography'].replace({"France": 0, "Spain": 1, "Germany": 2})
/var/folders/bj/4zbr51sj4fsbvtllcchl3wzc0000gn/T/ipykernel_1780/1114702664.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Gender'] = df['Gender'].replace({"Male": 1, "Female": 0})


In [40]:
X = df.drop(['Exited'], axis=1)
y = df.Exited

In [41]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)

In [46]:
from sklearn import tree 
clf_dt = tree.DecisionTreeClassifier(max_depth=1, random_state=0)

clf_dt.fit(X_train, y_train)
y_pred = clf_dt.predict(X_test)
print("Accuracy score of decision tree: {}".format(accuracy_score(y_test, y_pred)))

Accuracy score of decision tree: 0.999


In [44]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn import tree
from sklearn.metrics import accuracy_score

# Define the base classifier
base_clf = tree.DecisionTreeClassifier(max_depth=2)

# Initialize the AdaBoost classifier
adaboost = AdaBoostClassifier(estimator=base_clf, n_estimators=100)

# Fit the model
adaboost.fit(X_train, y_train)

# Make predictions
y_pred_ada = adaboost.predict(X_test)

# Print the accuracy score
print("Accuracy score of AdaBoost: {}".format(accuracy_score(y_test, y_pred_ada)))


/Users/mac/myenv/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Accuracy score of AdaBoost: 0.9985


In [45]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn import tree
from sklearn.metrics import accuracy_score

# Define the base classifier
base_clf = tree.DecisionTreeClassifier(max_depth=1)

# Initialize the AdaBoost classifier
adaboost = AdaBoostClassifier(estimator=base_clf, n_estimators=100)

# Fit the model
adaboost.fit(X_train, y_train)

# Make predictions
y_pred_ada = adaboost.predict(X_test)

# Print the accuracy score
print("Accuracy score of AdaBoost: {}".format(accuracy_score(y_test, y_pred_ada)))


/Users/mac/myenv/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Accuracy score of AdaBoost: 0.9985


In [48]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score

# Define the base models as a list of tuples
base_models = [
    ('knn', KNeighborsClassifier(n_neighbors=3)), 
    ('dt', DecisionTreeClassifier(max_depth=2, random_state=42))
]

# Define the meta-model
meta_model = LogisticRegression(max_iter=500, random_state=42)

# Initialize the stacking classifier
stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_model)

# Fit the model
stacking_model.fit(X_train, y_train)

# Make predictions
y_pred = stacking_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Stacking Accuracy: {accuracy:.2f}")


Stacking Accuracy: 1.00


In [49]:
for name, model in base_models:
    model.fit(X_train, y_train)
    y_pred_base = model.predict(X_test)
    accuracy_base = accuracy_score(y_test, y_pred_base)
    print(f"{name} Accuracy: {accuracy_base: .2f}")

knn Accuracy:  0.74
dt Accuracy:  1.00


In [50]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# Define the base models as a list of tuples
base_models = [
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)), 
    ('gb', GradientBoostingClassifier(n_estimators=100, random_state=42))
]

# Define the meta-model
meta_model = LogisticRegression(max_iter=500, random_state=42)

# Initialize the stacking classifier
stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_model)

# Fit the model
stacking_model.fit(X_train, y_train)

# Make predictions
y_pred = stacking_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Stacking Accuracy: {accuracy:.2f}")


Stacking Accuracy: 1.00


In [51]:
for name, model in base_models: 
    model.fit(X_train, y_train)
    y_pred_base = model.predict(X_test)
    accuracy_base = accuracy_score(y_test, y_pred_base)
    print(f'{name} Accuracy: {accuracy_base: .2f}')

rf Accuracy:  1.00
gb Accuracy:  1.00
